# KL Divergence

In this notebook I look at Kaggle's KL divergence metric and some alternate ways of calculating it.

## Kaggle

In [16]:
import numpy as np
import pandas as pd
import pandas.api.types
from typing import Optional

In [108]:
# This is Kaggle's function, see https://www.kaggle.com/code/metric/kullback-leibler-divergence/notebook
# AFAICT the arguments `micro_average` and `sample_weights` are not used for this competition.
def kl_divergence(solution: pd.DataFrame, submission: pd.DataFrame, epsilon: float, micro_average: bool, sample_weights: Optional[pd.Series]):
    # Overwrite solution for convenience
    for col in solution.columns:
        # Prevent issue with populating int columns with floats
        if not pandas.api.types.is_float_dtype(solution[col]):
            solution[col] = solution[col].astype(float)

        # Clip both the min and max following Kaggle conventions for related metrics like log loss
        # Clipping the max avoids cases where the loss would be infinite or undefined, clipping the min
        # prevents users from playing games with the 20th decimal place of predictions.
        submission[col] = np.clip(submission[col], epsilon, 1 - epsilon)

        y_nonzero_indices = solution[col] != 0
        solution[col] = solution[col].astype(float)
        solution.loc[y_nonzero_indices, col] = solution.loc[y_nonzero_indices, col] * np.log(solution.loc[y_nonzero_indices, col] / submission.loc[y_nonzero_indices, col])
        # Set the loss equal to zero where y_true equals zero following the scipy convention:
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.rel_entr.html#scipy.special.rel_entr
        solution.loc[~y_nonzero_indices, col] = 0

    if micro_average:
        return np.average(solution.sum(axis=1), weights=sample_weights)
    else:
        return np.average(solution.mean())

# Simplified version
def kl_div_simple(solution: np.ndarray, submission: np.ndarray, epsilon: float):
    return kl_divergence(pd.DataFrame({"votes": solution}),
                         pd.DataFrame({"votes": submission}),
                         epsilon, False, None)

In [109]:
# Example of usage
target = np.array([4, 0, 0, 2, 0, 0])
preds = np.array([0.91, 0.05, 0.04, 0, 0, 0])
print("Target:", target)
print("Say our model predicts:", preds)
print("First, clip everything to be strictly between 0 and 1.")
eps = 0.001
preds_clipped = np.clip(preds, eps, 1-eps)
print(preds_clipped)
print("Then, calculate P * log(P/Q), where P = target, Q = submission.")
print("This is set to 0 when P = 0.")
kl_arr = np.array([P*np.log(P/Q) if P != 0 else 0 for P, Q in zip(target, preds_clipped)])
print(kl_arr)
print("The KL divergence is the mean:")
print(kl_arr.mean())
print(kl_div_simple(target, preds, eps))

Target: [4 0 0 2 0 0]
Say our model predicts: [0.91 0.05 0.04 0.   0.   0.  ]
First, clip everything to be strictly between 0 and 1.
[0.91  0.05  0.04  0.001 0.001 0.001]
Then, calculate P * log(P/Q), where P = target, Q = submission.
This is set to 0 when P = 0.
[ 5.92242016  0.          0.         15.20180492  0.          0.        ]
The KL divergence is the mean:
3.5207041802414487
3.5207041802414487


## SciPy version

In [110]:
from scipy.stats import entropy
# Again, we have to clip to get a finite result
entropy(target, np.clip(preds, eps, 1-eps))



1.731940219993192

This differs from the Kaggle function in two ways:
1. SciPy normalizes both inputs to sum to 1.
2. SciPy takes the sum, rather than the mean.

In [111]:
target_norm = target / np.sum(target)
print("With target normalized:")
print(kl_div_simple(target_norm, preds, eps))
print("Summed rather than meaned:")
print(kl_div_simple(target_norm, preds, eps) * len(target))


With target normalized:
0.2881574518355656
Summed rather than meaned:
1.7289447110133938


SciPy's is slightly inaccurate -- not sure why.

## PyTorch version

This expects `preds` to be log-probabilities. NOT logits, as I'd said earlier! Logits are $\log \frac{p}{1-p}$. Here we just want $\log p$.

In [112]:
import torch
from torch.nn import KLDivLoss
loss_fn = KLDivLoss()
preds_log = np.log(preds_clipped)
preds_tensor = torch.tensor(preds_log)
targ_tensor = torch.tensor(target, dtype=torch.float64)  # Throws an error if handed ints
loss_fn(preds_tensor, targ_tensor).item()  # Note the argument order

/home/pvk/miniconda3/envs/brains/lib/python3.12/site-packages/torch/nn/functional.py:2949: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


3.5207041802414487

In [113]:
preds_tensor

tensor([-0.0943, -2.9957, -3.2189, -6.9078, -6.9078, -6.9078],
       dtype=torch.float64)

If our model output is `preds_tensor`, then we're expecting `exp(preds_tensor)` to be our vector of probabilities. This means that it's not meaningful for `preds_tensor` to have any nonnegative entries. If our final model layer is linear, I guess we should pass its output through an activation function that restricts it to the negative reals. It seems like a natural choice would be $\log \circ\, \mathrm{softmax}$ -- this would correspond to regarding our model outputs as logits.

A further complication with the PyTorch version is its behavior on batches. The default behavior (which the documentation calls incorrect) is to take the mean over the whole batch:

In [114]:
targ_batch = torch.stack([targ_tensor for _ in range(32)])
preds_batch = torch.stack([preds_tensor for _ in range(32)])
loss_fn(preds_batch, targ_batch)

tensor(3.5207, dtype=torch.float64)

The "correct" behavior is to just take the mean in the batch direction. This gives a result which is `len(target)` times as large:

In [115]:
loss_fn_batchmean = KLDivLoss(reduction="batchmean")
loss_fn_batchmean(preds_batch, targ_batch)

tensor(21.1242, dtype=torch.float64)

This discrepancy didn't appear in our first PyTorch example because we passed in 1D tensors. But if we do 2D tensors with a batch size of 1:

In [116]:
loss_fn_batchmean(preds_tensor.unsqueeze(0), targ_tensor.unsqueeze(0))

tensor(21.1242, dtype=torch.float64)

But what does Kaggle do?

In [117]:
kl_divergence(pd.DataFrame(targ_batch), pd.DataFrame(preds_batch.exp()), eps, False, None)

3.5207041802414487

## Takeaways

- We should use `torch.nn.KLDivLoss(reduction="mean")` and ignore the warning. This will calculate roughly the same thing as Kaggle, up to clipping.
- Model outputs should be constrained to be negative. I suggest a final activation function of $\log \circ\, \mathrm{softmax}, which keeps the final linear layer outputs interpretable as logits.
- The clipping in the Kaggle algorithm effectively forbids us from predicting a probability that's too low. Practically speaking, I think we don't need to use any corresponding clipping during training: A very small predicted probability will either result in a high loss penalty (if the target distribution has a nonzero probability there), or not contribute to either Kaggle's or our loss at all (if the target distribution is 0 there).